In [20]:
import pandas as pd
import openpyxl
import numpy as np

### 1. Load mapping

In [ ]:
sup1 = pd.read_excel("sample_dataset/41564_2018_306_MOESM3_ESM.xlsx", sheet_name=0, header=1)
print(sup1.columns)

feat_to_name = sup1[[
    "Metabolomic Feature",
    "Exact Match to Standard (* = isomer family)"
]].rename(columns={
    "Metabolomic Feature": "FeatureID",
    "Exact Match to Standard (* = isomer family)": "Metabolite"
})

Index(['Metabolomic Feature', 'Retention Time', 'm/z', 'Cluster (if DA)',
       'Putative Chemical Class',
       'Exact Match to Standard (* = isomer family)', 'Adduct'],
      dtype='object')


### 2. Load metabolites_to_SMILES and ChemBERTa Embeddings


In [23]:
met_to_smiles = pd.read_csv("metabolites_to_SMILES.csv")
met_to_smiles = met_to_smiles.rename(
    columns={"Exact Match to Standard (* = isomer family)": "Metabolite"}
)

chemberta = pd.read_csv("ChemBERTa_embeddings.csv")  # Metabolite, SMILES, emb_0..emb_767

feat_with_emb = (
    feat_to_name
    .merge(chemberta, on="Metabolite", how="inner")
)
print("Features with ChemBERTa embeddings:", feat_with_emb.shape)
feat_with_emb.head()

Features with ChemBERTa embeddings: (494, 771)


,FeatureID,Metabolite,SMILES,emb_0,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,...,emb_758,emb_759,emb_760,emb_761,emb_762,emb_763,emb_764,emb_765,emb_766,emb_767
0,C18-neg_Cluster_0183,"1,2,3,4-tetrahydro-b-carboline-1,3-dicarboxyli...",O=C(O)C1Cc2c([nH]c3ccccc23)C(C(=O)O)N1,2.404466,0.164168,0.066976,-0.580771,0.099839,-0.690932,0.120554,...,1.613474,0.000059,1.909015,0.675766,0.404137,0.615957,-0.638102,-1.055779,-1.206902,1.466978
1,C18-neg_Cluster_0530,13-docosenoate,CCCCCCCC/C=C\CCCCCCCCCCCC(=O)O,0.332203,0.113317,-0.579742,-0.236049,0.548618,-1.329615,-0.041671,...,0.237789,-0.163637,-0.337777,0.441929,1.319821,2.149394,0.072989,-1.365003,-0.513861,2.072217
2,HILIC-pos_Cluster_0245,1-methylguanine,Cn1c(N)nc2nc[nH]c2c1=O,0.091904,-1.092915,0.368594,-0.054166,-0.446534,-0.319817,-1.233743,...,0.405927,0.694337,0.661515,0.095675,0.055006,-0.442780,-0.616208,-1.550273,-0.955299,1.373021
3,HILIC-pos_Cluster_0110,1-methylnicotinamide,C[n+]1cccc(C(N)=O)c1,1.197902,0.082831,-0.382570,-2.521412,0.610822,-0.481389,-0.533924,...,-0.128521,0.388187,0.366318,-0.166581,-0.345590,0.156942,0.235276,-0.333513,-1.250880,1.083930
4,HILIC-neg_Cluster_0032,2-aminobutyric acid,CCC(N)C(=O)O,-0.188098,0.262202,-0.112233,-2.129169,0.068461,-1.396893,0.647778,...,0.073592,0.012995,0.522600,-0.917650,1.563296,0.612071,-0.887980,-1.789707,-0.501156,1.151684


### Load abundance matrix

We will load abundance matrix and get also the diagnosis to add the classifications for IBD vs Control (Binary) or UC vs CD vs Control (multiclass)

In [34]:
abundance = pd.read_excel("sample_dataset/41564_2018_306_MOESM4_ESM.xlsx", sheet_name=0, header=1)

# Rename first column for clarity
abundance = abundance.rename(columns={"# Feature / Sample": "FeatureID"})

print("Sup2 shape:", abundance.shape)
abundance.head()

Sup2 shape: (8855, 221)


,FeatureID,PRISM|7122,PRISM|7147,PRISM|7150,PRISM|7153,PRISM|7184,PRISM|7238,PRISM|7406,PRISM|7408,PRISM|7421,...,Validation|UMCGIBD00588,Validation|UMCGIBD00106,Validation|UMCGIBD00393,Validation|UMCGIBD00458,Validation|UMCGIBD00254,Validation|UMCGIBD00593,Validation|UMCGIBD00233,Validation|UMCGIBD00238,Validation|UMCGIBD00027,Validation|UMCGIBD00064
0,Age,38,50,41,51,68,67,59,52,58,...,68,42,65,44,71,21,32,38,51,43
1,Diagnosis,CD,CD,CD,CD,CD,CD,CD,CD,CD,...,UC,CD,UC,CD,CD,UC,CD,CD,CD,CD
2,Fecal.Calprotectin,207.484429,NaN,218.334517,NaN,20.167951,2.586247,NaN,NaN,79.331012,...,440,40,130,165,195,40,45,305,44,NaN
3,antibiotic,No,No,No,No,No,Yes,No,No,No,...,No,No,No,No,No,No,No,No,No,No
4,immunosuppressant,Yes,No,Yes,No,No,Yes,No,Yes,No,...,No,No,No,No,No,No,Yes,Yes,Yes,No


In [39]:
# Extract Diagnosis row in that same order
sample_ids = abundance.columns[1:]   # all columns except "FeatureID"
diag_row = abundance[abundance["FeatureID"] == "Diagnosis"]
diagnosis = diag_row.iloc[0, 1:].values   # skip FeatureID column
print(np.unique(diagnosis))

['CD' 'Control' 'UC']


In [37]:
# Filter abundance to features that have embeddings
abundance_filtered = abundance[abundance["FeatureID"].isin(feat_with_emb["FeatureID"])].copy()
print("Filtered Sup2 shape:", abundance_filtered.shape)

Filtered Sup2 shape: (256, 221)


### Note on handling metabolites annotation shared across multiple LC-MS features

If the goal is chemical-level analysis:
- Sum the abundances for all features annotated to the same metabolite.

If the goal is feature-level machine learning:

- Keep each feature, because each one has distinct patterns.

In [38]:
# Reorder abundance_filtered rows to match the order in feat_with_emb
abundance_filtered = abundance_filtered.set_index("FeatureID").loc[feat_with_emb["FeatureID"]].reset_index()

# Extract abundance matrix (rows = features, columns = samples)
sample_cols = abundance_filtered.columns[1:]   # all columns except FeatureID
A = abundance_filtered[sample_cols].to_numpy().astype(float)
print("A shape (features × samples):", A.shape)

# Embedding matrix (features × 768)
E = feat_with_emb.filter(like="emb_").to_numpy()
print("E shape (features × 768):", E.shape)


A shape (features × samples): (494, 220)
E shape (features × 768): (494, 768)


In [44]:
# Log-transform abundances to reduce skew (we can also implement z-score per metabolite and CLR TRANSFORMATION)
A_log = np.log1p(A)

# Normalize abundances per sample so columns sum to 1
weights = A_log / (A_log.sum(axis=0, keepdims=True) + 1e-8)

# Compute sample embeddings: (features × samples)^T @ (features × 768)
# Result: (samples × 768) <- should be 220 x 768
sample_embeddings = weights.T @ E 

print("Final sample embeddings shape:", sample_embeddings.shape)

Final sample embeddings shape: (220, 768)


### Build the dataset with emebeddings + classification labels

In [48]:
# Build dataframe with embeddings + labels
emb_df = pd.DataFrame(
    sample_embeddings,
    index=sample_ids,
    columns=[f"emb_{i}" for i in range(sample_embeddings.shape[1])]
)

emb_df.insert(0, "Diagnosis", diagnosis)
emb_df.insert(0, "SampleID", emb_df.index)

emb_df.head()


,SampleID,Diagnosis,emb_0,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,...,emb_758,emb_759,emb_760,emb_761,emb_762,emb_763,emb_764,emb_765,emb_766,emb_767
PRISM|7122,PRISM|7122,CD,1.385255,0.326943,-0.207165,-1.463018,-0.143149,-1.198966,-0.276479,0.109735,...,0.502403,-0.284824,0.963620,-0.896248,0.899836,0.243500,-0.807592,-1.048745,-0.458285,1.117722
PRISM|7147,PRISM|7147,CD,1.363603,0.355283,-0.178122,-1.503098,-0.150192,-1.199377,-0.326562,0.096814,...,0.490529,-0.309963,1.026563,-0.941518,0.832644,0.075253,-0.888930,-1.066275,-0.403023,1.074397
PRISM|7150,PRISM|7150,CD,1.388660,0.307955,-0.153669,-1.492275,-0.133827,-1.170884,-0.272714,0.087835,...,0.479314,-0.255009,0.985135,-0.877591,0.871247,0.174479,-0.854422,-1.088637,-0.439689,1.134031
PRISM|7153,PRISM|7153,CD,1.417473,0.442856,-0.260892,-1.471337,-0.239047,-1.246956,-0.336455,0.140689,...,0.529518,-0.337247,1.058742,-1.025205,0.894417,0.135408,-0.829306,-0.966675,-0.449452,1.036345
PRISM|7184,PRISM|7184,CD,1.384062,0.378006,-0.211944,-1.477190,-0.223676,-1.236622,-0.347293,0.133734,...,0.509494,-0.310017,1.043123,-0.945423,0.902847,0.174969,-0.791634,-1.029129,-0.453006,1.097607


### Save embeddings with samples

Note: for different transformations of abundances change name of data to keep track of transformations

In [51]:
emb_df.to_parquet("ChemBERTa_sample_embeddings_classification.parquet", index=False)